
# Create SiPM PSF for NEXT-100 (using detsim), to use with Beersheba. 

In [3]:
import os
import glob
import numpy  as np
import pandas as pd
import tables as tb
import h5py

from IC.invisible_cities.reco.psf_functions    import create_psf
from IC.invisible_cities.reco.psf_functions    import hdst_psf_processing
from IC.invisible_cities.reco.psf_functions    import add_empty_sensors_and_normalize_q
from IC.invisible_cities.reco.psf_functions    import add_variable_weighted_mean

import IC.invisible_cities.core.core_functions as     coref
import IC.invisible_cities.io.dst_io         as     dstio

from IC.invisible_cities.database              import load_db
from IC.invisible_cities.io.kdst_io      import psf_writer

Note: I had to change the path for the database files in invisible cities, because it was calling an error. 

In [4]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"]          = 10, 10
plt.rcParams["font.size"]               = 16
plt.rcParams["figure.max_open_warning"] = 100

In [5]:
db = load_db.DataSiPM('next100', -1)

In [6]:
# Input and output path
psf_path   = '/Users/halmamol/NEXT/files/NEXT100/psf/'
psf_filename = 'next100.kr83m.psf.h5'
out_psf  = psf_path + psf_filename

psf_files = glob.glob(psf_path+'psf_*.h5')

*Below*: When I call the files, it sais that the first one is the 24 but there is a file 1.


In [ ]:
psf_files[0]

### 1-Check if penthesilea (kr_penthesilea) files are correct

Opening file, to retrieve important variables storaged in RECO/Events (X and Y rms, charge, energy, trackID):

In [ ]:
myfile = ['/Users/halmamol/NEXT/files/NEXT100/psf/kr_penthesilea/next100.kr83m.45.psf_hits.h5']
df = dstio.load_dsts(myfile, 'RECO', 'Events').drop(['Xrms', 'Yrms', 'Qc', 'Ec', 'track_id'], 
                                                    axis='columns').reset_index(drop=True)

In [ ]:
### Use all z to have more statistics
plt.rcParams["figure.figsize"]  = 10, 10

bin_size   = 1
Xrange     = [ -100, 100]
Yrange     = [ -100, 100]
ranges     = [Xrange, Yrange]
nbinX      = int(np.diff(Xrange)/bin_size)
nbinY      = int(np.diff(Yrange)/bin_size)

# "hdst_psf_processing" fucntion: adds necessary info to a hits DST to create the PSF, 
# namely the relative position and the normalized Q.

hdstA = hdst_psf_processing(df, [Xrange, Yrange], db)
hdst = hdstA[coref.in_range(hdstA.Z, 0, 2000)]
bin_edges = [np.linspace(*rr, [nbinX, nbinY][i]+1) for i, rr in enumerate([Xrange, Yrange])]

# "create_psf" function takes:
# - the hits relative position in 2D (hdst.RelX.values, hdst.RelY.values), 
# - the hits SiPM charge normalized to the total peak charge (hdst.NormQ)
# - bin edges for the PSF in each dimension (bin_edges)

# and it retrieves:
# - the point-spread function, 
# - number of entries per bin in the PSF, 
# - bin centers of the PSF.

psf_new, entries_new, bins_new = create_psf((hdst.RelX.values, hdst.RelY.values), hdst.NormQ, 
                                             bin_edges)
plt.imshow(psf_new)
plt.colorbar()

### 2-Code to create PSF 

In [ ]:
## Defining "compute_psf" function:
## - It creates a test file ("prova")
## - It uses one big bin in xy.
## - In the writer function, 0., 0. are the xy average of the particular xy bin.
## - Other two loops should be added if more bins were to be used.

def compute_psf(df, fnumber):
    out_psf = psf_path + 'psf_prova_{}.h5'.format(fnumber)
#    print(out_psf)

    with tb.open_file(out_psf, 'w') as outfile:
        # Declare the PSF writer
        writer = psf_writer(outfile)

        for z in zrange:
            z_sel = coref.in_range(df.Z, z, z+z_step) 
            # Preprocess the data before extracting the PSF (same as before)
            hdst = hdst_psf_processing(df[z_sel], [Xrange, Yrange], the_db)
            # Safety check (single sipm events not considered to be physical)
            hdst = hdst[hdst.nsipm > 1]

            # Loop to extract the PSF in different regions.
            bin_edges = [np.linspace(*rr, [nbinX, nbinY][i]+1) for i, rr in enumerate([Xrange, Yrange])]
            psf_new, entries_new, bins_new = create_psf((hdst.RelX.values, hdst.RelY.values), hdst.NormQ, 
                                                         bin_edges)

            writer(bins_new[0], bins_new[1], [0], 0., 0., z + z_step/2, 
                           np.asarray([psf_new]).transpose((1, 2, 0)), np.asarray([entries_new]).transpose((1, 2, 0)))

In [ ]:
# Calling the SiPM data from the db
run = -1
the_db = load_db.DataSiPM('next100', run)

z_max  = 1205
z_step = 1205
zrange = []
zrange.extend(np.arange(0, z_max, z_step))

In [ ]:
## Checking db is working
the_db

In [ ]:
### Definign PSF binning and range (as before)
bin_size   = 1
Xrange     = [ -100, 100]
Yrange     = [ -100, 100]
ranges     = [Xrange, Yrange]
nbinX      = int(np.diff(Xrange)/bin_size)
nbinY      = int(np.diff(Yrange)/bin_size)

#Taking the variables from the files
i = 1
thefile = myfile[0].format(i)
df = dstio.load_dsts([thefile], 'RECO', 'Events').drop(['Xrms', 'Yrms', 'Qc', 'Ec', 'track_id'], axis='columns').reset_index(drop=True)
compute_psf(df, i)

In [ ]:
## Test - Crosscheck
plt.imshow(psf_new)
plt.colorbar()

### 3 - Read back the psf (prova) file

In [ ]:
provapath = psf_path + 'psf_prova_1.h5'
h5file = tb.open_file(psf_path + 'psf_prova_1.h5')
h5file

In [ ]:
## Take one of the prova files - NOTE: I had to add 'PSF/PSFs'
psf = pd.read_hdf(psf_path + 'psf_prova_1.h5', 'PSF/PSFs')
print(psf.head())

In [ ]:
#Checking the psf form
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

### 1s - Plot PSF at center (X dimension)
## Taking a specific z and yr value
p1 = psf[(psf.z==602.5) & (psf.yr==0.5)]
x, y = sorted(set(p1.xr)), p1.groupby(['xr'])['factor'].sum().values
axes[0].plot(x, y)
axes[0].set_xlabel('X distance (mm)')
axes[0].set_ylabel('Charge fraction')

### PSF (2D)
p1 = psf[(psf.z==602.5)]
x, y, e = p1.xr.values, p1.yr.values, p1.factor.values
xb, yb  = np.linspace(Xrange[0], Xrange[1], Xrange[1]), np.linspace(Yrange[0], Yrange[1], Yrange[1])
img = axes[1].hist2d(x, y, bins=[xb,yb], weights=e)
axes[1].set_xlabel('X distance (mm)')
axes[1].set_ylabel('Y distance (mm)')
cb = fig.colorbar(img[3], ax=axes[1])

cb.set_label('Charge fraction')

# Join all PSFs

Now, once all psf files have been created, let's join them to use one file.

In [ ]:
# Input and output path

psf_filename = 'next100.kr83m.psf.h5'
out_psf  = psf_path + psf_filename

psf_files = glob.glob(psf_path + 'psf_*.h5')
psf_files

In [ ]:
len(psf_files)

It seems some of the files are missing, maybe when moving the files into local

In [10]:
df_psf = pd.read_hdf(psf_files[0], 'PSF/PSFs')
df_psf

,nevt,xr,yr,zr,x,y,z,factor
0,0,-99.5,-99.5,0.0,0.0,0.0,10.0,0.0
1,0,-99.5,-98.5,0.0,0.0,0.0,10.0,0.0
2,0,-99.5,-97.5,0.0,0.0,0.0,10.0,0.0
3,0,-99.5,-96.5,0.0,0.0,0.0,10.0,0.0
4,0,-99.5,-95.5,0.0,0.0,0.0,10.0,0.0
...,...,...,...,...,...,...,...,...
2439995,0,99.5,95.5,0.0,0.0,0.0,1210.0,0.0
2439996,0,99.5,96.5,0.0,0.0,0.0,1210.0,0.0
2439997,0,99.5,97.5,0.0,0.0,0.0,1210.0,0.0
2439998,0,99.5,98.5,0.0,0.0,0.0,1210.0,0.0


In [7]:
evts    = []
factors = []

for i in range(1, 1000):
    filein = psf_path + f'psf_{i}.h5'
    try:
        df = pd.read_hdf(filein, 'PSF/PSFs')
    except:
        print(f'{filein} not found')
        continue
        
    if len(df.nevt.values) != 2440000:
        print(i, len(df.nevt.values))
    else:
        evts   .append(df.nevt  .values)
        factors.append(df.factor.values)

#for i, filein in enumerate(psf_files):
#    if i % 10 == 0:
#        print(f'{i} files read, current file: {filein}')
#    df = pd.read_hdf(filein)
#    evts   .append(df.nevt  .values)
#    factors.append(df.factor.values)

4 0
/Users/halmamol/NEXT/files/NEXT100/psf/psf_5.h5 not found
6 0
/Users/halmamol/NEXT/files/NEXT100/psf/psf_7.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_8.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_9.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_10.h5 not found
13 0
/Users/halmamol/NEXT/files/NEXT100/psf/psf_14.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_15.h5 not found
16 0
/Users/halmamol/NEXT/files/NEXT100/psf/psf_17.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_18.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_19.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_20.h5 not found
211 2400000
/Users/halmamol/NEXT/files/NEXT100/psf/psf_212.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_213.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_214.h5 not found
/Users/halmamol/NEXT/files/NEXT100/psf/psf_215.h5 not found
276 0
/Users/halmamol/NEXT/files/NEXT100/psf/psf_277.h5 not found
/Users/halmamol/NEX

In [8]:
#It seems I didn't get the last 100 files, and some of them present some error:
len(evts), len(factors)

(946, 946)

In [ ]:
evts_all    = np.array(evts)
factors_all = np.array(factors)
new_evts = evts_all.sum(axis=0).astype('int32')
tmp_factors = (evts_all * factors_all).sum(axis=0)
new_factors = tmp_factors

In [11]:
df_psf.head()

,nevt,xr,yr,zr,x,y,z,factor
0,0,-99.5,-99.5,0.0,0.0,0.0,10.0,0.0
1,0,-99.5,-98.5,0.0,0.0,0.0,10.0,0.0
2,0,-99.5,-97.5,0.0,0.0,0.0,10.0,0.0
3,0,-99.5,-96.5,0.0,0.0,0.0,10.0,0.0
4,0,-99.5,-95.5,0.0,0.0,0.0,10.0,0.0


In [12]:
df_psf.dtypes

nevt       uint32
xr        float32
yr        float32
zr        float32
x         float32
y         float32
z         float32
factor    float32
dtype: object

In [13]:
evts_all    = np.array(evts)
factors_all = np.array(factors)

In [ ]:
evts_all

In [14]:
new_evts = evts_all.sum(axis=0).astype('int32')

In [ ]:
new_evts

In [ ]:
factors_all

In [ ]:
tmp_factors = (evts_all * factors_all).sum(axis=0)

In [ ]:
new_factors = tmp_factors

In [ ]:
new_factors

In [ ]:
df_psf.nevt   = new_evts
df_psf.factor = new_factors

In [ ]:
df_psf.head()

In [ ]:
df_psf.dtypes

### Write the file in the correct format¶

In [ ]:
from IC.invisible_cities.evm.nh5     import PSFfactors
from IC.invisible_cities.io.table_io import make_table

with tb.open_file(out_psf, 'w') as outfile:
    psf_table = make_table(outfile,
                           group       = "PSF",
                           name        = "PSFs",
                           fformat     = PSFfactors,
                           description = "XYZ dependent point spread functions",
                           compression = 'ZLIB4')

    row = psf_table.row
    for xr, yr, zr, x, y, z, f, ne in zip(df_psf.xr.values, df_psf.yr.values, df_psf.zr.values, 
                                          df_psf.x.values, df_psf.y.values, df_psf.z.values, 
                                          df_psf.factor.values, df_psf.nevt.values):
        row["xr"    ] = xr
        row["yr"    ] = yr
        row["zr"    ] = zr
        row["x"     ] = x
        row["y"     ] = y
        row["z"     ] = z
        row["factor"] = f
        row["nevt"  ] = ne
        row.append()

### Read the PSF

In [1]:
h5file = tb.open_file(psf_path + 'next100.kr83m.psf.h5')
h5file

NameError: name 'tb' is not defined

In [ ]:
psf = pd.read_hdf(psf_path + 'next100.kr83m.psf.h5', 'PSF/PSFs')
psf.head()

In [ ]:
psf =  dstio.load_dst(out_psf, 'PSF', 'PSFs')
plt.hist(psf.z, bins=50);

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

bin_size   = 1
Xrange     = [ -100, 100]
Yrange     = [ -100, 100]
ranges     = [Xrange, Yrange]
nbinX      = int(np.diff(Xrange)/bin_size)
nbinY      = int(np.diff(Yrange)/bin_size)

### Plot PSF at center (X dimension)
p1 = psf[(psf.z==610.) & (psf.yr==0.5)]
x, y = sorted(set(p1.xr)), p1.groupby(['xr'])['factor'].sum().values
axes[0].plot(x, y)
axes[0].set_xlabel('X distance (mm)')
axes[0].set_ylabel('Charge fraction')

### PSF (2D)
p1 = psf[(psf.z==610.)]
x, y, e = p1.xr.values, p1.yr.values, p1.factor.values
xb, yb  = np.linspace(Xrange[0], Xrange[1], Xrange[1]), np.linspace(Yrange[0], Yrange[1], Yrange[1])
img = axes[1].hist2d(x, y, bins=[xb,yb], weights=e)
axes[1].set_xlabel('X distance (mm)')
axes[1].set_ylabel('Y distance (mm)')
axes[1].set_xlim(-50, 50)
axes[1].set_ylim(-50, 50)
cb = fig.colorbar(img[3], ax=axes[1])

cb.set_label('Charge fraction')